# [CSC8101] Engineering for AI - 2024 Spark Coursework

## Coursework overview

### Inputs

- **NYC Taxi Trips dataset** - list of recorded taxi trips, each with several characteristics, namely: distance, number of passengers, origin zone, destination zone and trip cost (total amount charged to customer).
- **NYC Zones dataset** - list of zones wherein trips can originate/terminate.

### Tasks

1. Data cleaning
  1. Remove "0 distance" and 'no passengers' records.
  2. Remove outlier records. 
2. Add new columns
  1. Join with zones dataset
  2. Compute the unit profitability of each trip
3. Zone summarisation and ranking
  1. Summarise trip data per zone
  2. Obtain the top 10 ranks according to:
    1. The total trip volume
    2. Their average profitabilitiy
    3. The total passenger volume
4. Record the total and task-specific execution times for each dataset size and format.

### How to

###### Code structure and implementation

- You must implement your solution to each task in the provided function code skeleton.
- The task-specific functions are combined together to form the full pipeline code, executed last (do not modify this code).
- Before implementing the specified function skeleton, you should develop and test your solution on separate code cells (create and destroy cells as needed).

###### Development

- Develop an initial working solution for the 'S' dataset and only then optimise it for larger dataset sizes.
- To perform vectorised operations on a DataFrame:
  - use the API docs to look for existing vectorised functions in: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/functions.html
  - actions to get around the lazy execution of spark: 
  https://spark.apache.org/docs/latest/rdd-programming-guide.html#actions
  - if a customised function is required (e.g. to add a new column based on a linear combination of other columns), implement your own User Defined Function (UDF). See:  https://spark.apache.org/docs/latest/sql-ref-functions-udf-scalar.html
- Use only the `pyspark.sql` API - documentation link below - (note that searching through the docs returns results from the `pyspark.sql` API together with the `pyspark.pandas` API):
  - https://spark.apache.org/docs/3.2.0/api/python/reference/pyspark.sql.html
- Periodically download your notebook to your computer as backup and safety measure against accidental file deletion.
 
###### Execution time measurement

- Execution time is calculated and returned by the Spark Engine and shown in the output region of the cell.
- To measure the execution time of a task you must perform a `collect` or similar operation (e.g. `take`) on the returned DataFrame.

## Task 0 - Read data

The code below is ready to run. **Do not modify this code**. It does the following:

- Reads the 'zones' dataset into variable 'zone_names'
- Defines the `init_trips` function that allows you to read the 'trips' dataset (from the DBFS FileStore) given the dataset size ('S' to 'XXL') and format ('parquet' or 'delta') as function arguments
- Defines the `pipeline` function, called in Task 4 to measure the execution time of the entire data processing pipeline
- Shows you how to call the `init_trips` function and display dataset characteristics (number of rows, schema)

In [0]:
## global imports
import pyspark.sql as ps
import pyspark.sql.functions as pf
import pandas as pd

# Load zone names dataset - (much faster to read small file from git than dbfs)
zones_file_url = 'https://raw.githubusercontent.com/mutazb999/CSC8101-lab-and-coursework/main/02-assignment-spark/taxi_zone_names.csv'
zone_names = spark.createDataFrame(pd.read_csv(zones_file_url))

# Function to load trips dataset by selected dataset size
def init_trips(size = 'S', data_format = "parquet", taxi_folder = "/FileStore/tables/taxi"):     
    
    files = {
        'S'  : ['2021_07'],
        'M'  : ['2021'],
        'L'  : ['2020_21'],
        'XL' : ['1_6_2019', '7_12_2019'],
        'XXL': ['1_6_2019', '7_12_2019', '2020_21']
    }
    
    # validate input dataset size
    if size not in files.keys():
        print("Invalid input dataset size. Must be one of {}".format(list(files.keys())))
        return None               
    
    if data_format == "parquet":
        filenames = list(map(lambda s: f'{taxi_folder}/parquet/tripdata_{s}.parquet', files[size]))
        trips_df = spark.read.parquet(filenames[0])
        
        for name in filenames[1:]:
            trips_df = trips_df.union(spark.read.parquet(name))
            
    elif data_format == "delta":
        filenames = f"{taxi_folder}/delta/taxi-{size}-delta/"
        trips_df = spark.read.format("delta").load(filenames)
    
    else:
        print("Invalid data format. Must be one of {}".format(['parquet', 'delta']))
        return None
        
    print(
    """
    Trips dataset loaded!
    ---
      Size: {s}
      Format: {f}
      Tables loaded: {ds}
      Number of trips (dataset rows): {tc:,}
    """.format(s = size, f = data_format, ds = filenames, tc = trips_df.count()))
    
    return trips_df

# helper function to print dataset row count
def print_count(df):
    print("Row count: {t:,}".format(t = df.count()))

def pipeline(trips_df, with_task_12 = False, zones_df = zone_names):
    # Do not edit
    #---

    ## Task 1.1
    _trips_11 = t11_remove_zeros(trips_df)

    ## Task 1.2
    if with_task_12:
        _trips_12 = t12_remove_outliers(_trips_11)
    else:
        _trips_12 = _trips_11

    ## Task 2.1
    _trips_21 = t21_join_zones(_trips_12, zones_df = zone_names)

    ## Task 2.2
    _trips_22 = t22_calc_profit(_trips_21)

    ## Task 3.1
    _graph = t31_summarise_trips(_trips_22)

    ## Task 3.2
    _zones = t32_summarise_zones_pairs(_graph)

    _top10_trips     = t32_top10_trips(_zones)
    _top10_profit    = t32_top10_profit(_zones)
    _top10_passenger = t32_top10_passenger(_zones)
    
    return([_top10_trips, _top10_profit, _top10_passenger])

In [0]:
# CHANGE the value of argument 'size' to record the pipeline execution times for increasing dataset sizes
SIZE = 'S'
DATA_FORMAT = 'parquet'

# Load trips dataset
trips = init_trips(SIZE, DATA_FORMAT)

# uncomment line only for small datasets
# trips.take(1)


    Trips dataset loaded!
    ---
      Size: S
      Format: parquet
      Tables loaded: ['/FileStore/tables/taxi/parquet/tripdata_2021_07.parquet']
      Number of trips (dataset rows): 2,898,033
    


In [0]:
print_count(trips)

Row count: 2,898,033


In [0]:
# dataset schemas
trips.printSchema()

root
 |-- index: long (nullable = true)
 |-- VendorID: double (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- cab_type: string (nullable = true)
 |-- lpep_pickup_datetime: string (nullable = true)
 |-- lpep_dropoff_datetime: string (nullable = true)
 |--

In [0]:
display(trips[['PULocationID', 'DOLocationID', 'trip_distance', 'passenger_count', 'total_amount']].take(5))

PULocationID,DOLocationID,trip_distance,passenger_count,total_amount
90,68,0.8,1.0,8.8
113,90,0.9,1.0,8.8
88,232,2.8,1.0,13.8
79,249,1.4,1.0,12.3
142,238,2.0,0.0,12.3


In [0]:
zone_names.printSchema()

root
 |-- LocationID: long (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [0]:
display(zone_names.take(5))

LocationID,Borough,Zone,service_zone
1,EWR,Newark Airport,EWR
2,Queens,Jamaica Bay,Boro Zone
3,Bronx,Allerton/Pelham Gardens,Boro Zone
4,Manhattan,Alphabet City,Yellow Zone
5,Staten Island,Arden Heights,Boro Zone


## Task 1 - Filter rows

**Input:** trips dataset

### Task 1.1 - Remove "0 distance" and 'no passengers' records

Remove dataset rows that represent invalid trips:

- Trips where `trip_distance == 0` (no distance travelled)
- Trips where `passenger_count == 0` and `total_amount == 0` (we want to retain records where `total_amount` > 0 - these may be significant as the taxi may have carried some parcel, for example)

Altogether, a record is removed if it satisfies the following conditions:

`trip_distance == 0` or `(passenger_count == 0` and `total_amount == 0)`.

**Recommended:** Select only the relevant dataset columns for this and subsequent tasks: `['PULocationID', 'DOLocationID', 'trip_distance', 'passenger_count', 'total_amount')]`

### Task 1.2 - Remove outliers using the modified z-score

Despite having removed spurious "zero passengers" trips in task 1.1, columns `total_amount` and `trip_distance` contain additional outlier values that must be identified and removed.

To identify and remove outliers, you will use the modified [z-score](https://en.wikipedia.org/wiki/Standard_score) method.
The modified z-score uses the median and [Median Absolute Deviation](https://en.wikipedia.org/wiki/Median_absolute_deviation) (MAD), instead of the mean and standard deviation, to determine how far an observation (indexed by i) is from the mean:

$$z_i = \frac{x_i - \mathit{median}(\mathbf{x})}{\mathbf{MAD}},$$

where x represents the input vector, xi is an element of x and zi is its corresponding z-score. In turn, the MAD formula is:

$$\mathbf{MAD} = 1.483 * \mathit{median}(\big\lvert x_i - \mathit{median}(\mathbf{x})\big\rvert).$$

Observations with **high** (absolute) z-score are considered outlier observations. A score is considered **high** if its __absolute z-score__ is larger than a threshold T = 3.5:

$$\big\lvert z_i \big\rvert > 3.5.$$

where T represents the number of unit standard deviations beyond which a score is considered an outlier ([wiki](https://en.wikipedia.org/wiki/68%E2%80%9395%E2%80%9399.7_rule)).

This process is repeated twice, once for each of the columns `total_amount` and `trip_distance` (in any order).

**Important:** Use the surrogate function [`percentile_approx`](https://spark.apache.org/docs/3.2.0/api/python/reference/api/pyspark.sql.functions.percentile_approx.html?highlight=percentile#pyspark.sql.functions.percentile_approx) to estimate the median (calculating the median values for a column is expensive as it cannot be parallelised efficiently).

In [0]:
# develop your solution here (create/destroy cells as needed) and then implement it in the functions below


##### Task 1.1

The function **t11_remove_zeros**, provided below, operates on a dataframe. Initially, it extracts specified column names such as *'PULocationID', 'DOLocationID', 'trip_distance', 'passenger_count', 'total_amount'*. Subsequently, it eliminates rows where *trip_distance* equals zero (0) or where *passenger_count* equals zero only when *total_amount* exceeds zero (indicating cargo transportation with a charged amount despite zero passengers). Furthermore, the function filters out rows where both *passenger_count* and *total_amount* are zero.

In [0]:
# Your solution implementation to task 1.1 goes HERE
def t11_remove_zeros(df):
    # input: trips dataset
    df = trips.select('PULocationID', 'DOLocationID', 'trip_distance', 'passenger_count', 'total_amount')
    df = df.filter(
        (df.trip_distance != 0) & 
        (
            ((df.total_amount > 0) & (df.passenger_count == 0)) | ((df.passenger_count != 0) & (df.total_amount != 0))
        )
    )
    return df

In [0]:
# execute task 1.1
trips_11 = t11_remove_zeros(trips)

print_count(trips_11)

## uncomment only for smaller datasets
# display(trips_11.take(10))

Row count: 2,693,786



##### Task 1.2

The following function operates on a dataframe to *filter out outliers*. Initially, it computes the *median values for trip distance and total amount*. Then two additional columns are created: *"total_amount_median_diff" and "trip_distance_median_diff",* capturing the difference between the actual values and their respective medians. Moving forward, **Median Absolute Deviation (MAD)** is calculated for these differences (by using *percentile_approx*), denoted as *total_amount_MAD* and *trip_distance_MAD*, correspondingly.*Modified Z-Scores* are computed for *trip distance* and *total amount* using the provided formula. Finally, data is filtered to retain only records where **the Z-Score for both trip distance and total amount is less than or equal to 3.5**.

In [0]:
# Your solution implementation to task 1.2 goes HERE
def t12_remove_outliers(df):
    # Calculation of median values for trip distance and total amount:
    medians = df.select(pf.percentile_approx("trip_distance", 0.5).alias('trip_distance_median'),pf.percentile_approx("total_amount", 0.5).alias('total_amount_median')).collect()[0]

    # Assigning median values to trip_distance_median, total_amount_median:
    trip_distance_median = medians[0]
    total_amount_median = medians[1]

    # Difference of actual value and median of it:
    df = df.withColumn('total_amount_median_diff', pf.round(pf.abs(pf.col('total_amount') - total_amount_median), 2))
    df = df.withColumn('trip_distance_median_diff', pf.round(pf.abs(pf.col('trip_distance') - trip_distance_median), 2))

    # Calculation of the median absolute deviation (MAD) for total amount and trip distance:
    MAD = df.select(pf.percentile_approx("total_amount_median_diff", 0.5).alias('total_amount_MAD'),pf.percentile_approx("trip_distance_median_diff", 0.5).alias('trip_distance_MAD') ).collect()[0]

    # Assigning MAD values to total_amount_MAD, trip_distance_MAD:
    total_amount_MAD = MAD[0]
    trip_distance_MAD = MAD[1]
    
    # Calculate Modified Z-score using the formula provided above (in task description)
    df = df.withColumn('Z_score_trip_distance', pf.abs(pf.round((pf.col('trip_distance') - trip_distance_median) / (1.438 * trip_distance_MAD ),2 )))
    df = df.withColumn('Z_score_total_amount', pf.abs(pf.round((pf.col('total_amount') - total_amount_median) / (1.438 * total_amount_MAD ),2 )))
    
    # Filtering only those data where Z score is less than or equal to 3.5:
    df = df.filter(pf.col('Z_score_total_amount') <= 3.5)
    df = df.filter(pf.col('Z_score_trip_distance') <= 3.5)
    df = df.select('PULocationID', 'DOLocationID', 'trip_distance', 'passenger_count', 'total_amount')

    return df

In [0]:
# execute task 1.2
trips_12 = t12_remove_outliers(trips_11)

print_count(trips_12)
# display(trips_12.take(10))

Row count: 2,365,045


## Task 2 - Compute new columns

### Task 2.1 - Zone names

Obtain the **start** and **end** zone names of each trip by joining the `trips` and `zone_names` datasets (i.e. by using the `zone_names` dataset as lookup table).

**Note:** The columns containing the start and end zone ids of each trip are named `PULocationID` and `DOLocationID`, respectively.

### Task 2.2 - Unit profitability

Compute the column `unit_profitability = total_amount / trip_distance`.

In [0]:
# develop your solution here (create/destroy cells as needed) and then implement it in the functions below


##### Task 2.1

The following function takes an argument dataframe (which is the output of task 1.2) and zone names dataset. Later, I've specified list of two columns *"["PULocationID", "DOLocationID"]"* then I've looped through these columns and ***joined the trips data with zone names data***. Lastly, I've dropped certain columns which are not required.

In [0]:
# Your solution implementation to task 2.1 goes HERE
def t21_join_zones(df, zones_df = zone_names):
    # input: output of task 1.2 and zone_names dataset
    columns = ["PULocationID", "DOLocationID"]
    # Joining trips with zones
    for column in columns:
        column_zone = column +'_zone'
        df = (df.join(zone_names,(pf.col(column) == zones_df.LocationID),"inner").drop("LocationID", "Borough", "service_zone").withColumnRenamed("zone",column_zone))

    return df

In [0]:
# execute task 2.1
trips_21 = t21_join_zones(trips_12, zones_df = zone_names)

print_count(trips_21)
# display(trips_21.take(10))

Row count: 2,365,045



##### Task 2.2

The below function **computes the unit profitability** using the *formula: total_amount / trip_distance*, and storing the result into the *unit_profitability* columns. The results are rounded to 2 decimal places.

In [0]:
# Your solution implementation to task 2.2 goes HERE
def t22_calc_profit(df):
    # input: output of task 2.1
    # Calculation of unit profitability:
    df = df.withColumn("unit_profitability", pf.round((df.total_amount / df.trip_distance), 2))
    return df

In [0]:
# execute task 2.2
trips_22 = t22_calc_profit(trips_21)

print_count(trips_22)
# display(trips_22.take(10))

Row count: 2,365,045


## Task 3: Rank zones by traffic, passenger volume and profitability

### 3.1 - Summarise interzonal travel

Build a graph data structure of zone-to-zone traffic, representing aggregated data about trips between any two zones. The graph will have one node for each zone and one edge connecting each pair of zones. In addition, edges contain aggregate information about all trips between those zones. 

For example, zones Z1 and Z2 are connected by *two* edges: edge Z1 --> Z2 carries aggregate data about all trips that originated in Z1 and ended in Z2, and edge Z2 --> Z1 carries aggregate data about all trips that originated in Z2 and ended in Z1.

The aggregate information of interzonal travel must include the following data:

- `average_unit_profit` - the average unit profitability (calculated as `mean(unit_profitability)`).
- `trips_count` -- the total number of recorded trips.
- `total_passengers` -- the total number of passenger across all trips (sum of `passenger_count`).

This graph can be represented as a new dataframe, with schema:

\[`PULocationID`, `DOLocationID`, `average_unit_profit`, `trips_count`, `total_passengers` \]

__hint__: the `groupby()` operator produces a `pyspark.sql.GroupedData` structure. You can then calculate multiple aggregations from this using `pyspark.sql.GroupedData.agg()`: 
- https://spark.apache.org/docs/3.2.0/api/python/reference/pyspark.pandas/api/pyspark.pandas.DataFrame.groupby.html
- https://spark.apache.org/docs/3.2.0/api/python/reference/api/pyspark.sql.GroupedData.agg.html

### Task 3.2 - Obtain top-10 zones

For each of the following measures, report the top-10 zones _using their plain names you dereferenced in the previous step, not the codes_. Note that this requires ranking the nodes in different orders. Specifically, you need to calculate the following further aggregations:

- the **total** number of trips originating from Z. This is simply the sum of `trips_count` over all outgoing edges for Z, i.e., edges of the form Z -> \*
- the **average** profitability of a zone. This is the average of all `average_unit_profit` over all *outgoing* edges from Z.
- The **total** passenger volume measured as the **sum** of `total_passengers` carried in trips that originate from Z

In [0]:
# develop your solution here (create/destroy cells as needed) and then implement it in the functions below


##### Task 3.1

The function below accepts a DataFrame as an argument. It proceeds by grouping the DataFrame according to *PULocationID* and *DOLocationID*. In the initial DataFrame, named "df1", the **total number of trips** is tallied based on these locations. Subsequently, leveraging the `pf.mean` function, the **average unit profitability** and **total number of passengers** are computed for each location. Finally, the results from these two separate DataFrames are merged using the `join` function.

In [0]:
## Your solution to task 3.1 goes HERE
def t31_summarise_trips(df):
    # input: output of task 2.2
    # Grouping dataframe based on PULocationID and DOLocationID, then performing required operations:
    df1=df.groupby(["PULocationID", "DOLocationID"]).count().withColumnRenamed("count", "trips_count")
    df2=df.groupby(["PULocationID", "DOLocationID"]).agg(pf.mean("unit_profitability").alias("average_unit_profit"),
                                                              pf.sum("passenger_count").alias("total_passengers"))
    df=df1.join(df2,["PULocationID", "DOLocationID"])
    return df

In [0]:
# execute task 3.1
graph = t31_summarise_trips(trips_22)

print_count(graph)
# display(graph.take(10))

Row count: 9,857



##### Task 3.2

The **main task of this section is to get top 10 ranked zones based on three different independent criterias**. The first function, named *t32_summarise_zones_pairs*, **associates zone IDs with their corresponding zone names** and returns the zone data. This zone data is then passed to the second function (*t32_top10_trips*) in order to achieve first criteria, responsible for **identifying the top 10 zones ranked by trip volume**. Subsequently, the same zone data are inputted into the third function (*t32_top10_profit*), which determines the **top 10 zones ranked by average unit profits**. Finally, the last function (*t32_top10_passenger*) calculates the **top 10 zones ranked by total passengers** for each zone.

In [0]:
# Your solution to task 3.2 goes HERE (implement each of the functions below)
def t32_summarise_zones_pairs(df, zones_df = zone_names):
    columns = ["PULocationID", "DOLocationID"]
    # Joining trips with zones
    for column in columns:
        column_zone = column +'_zone'
        df = (df.join(zone_names, (pf.col(column) == zones_df.LocationID),"inner").drop("LocationID", "Borough", "service_zone").withColumnRenamed("zone",column_zone))
    return df

# Top 10 ranked zones by traffic (trip volume)
def t32_top10_trips(df_zones):
    # input: output of task 3.2
    df_zones = zones.groupby(["PULocationID_zone"]).agg(pf.sum("trips_count").alias("trips")).sort(["trips"], ascending = False)
    return df_zones.collect()[:10:]

# Top 10 ranked zones by profit
def t32_top10_profit(df_zones):
    # input: output of task 3.2
    df_zones = zones.groupby(["PULocationID_zone"]).agg(pf.avg("average_unit_profit").alias("average")).sort(["average"], ascending = False)
    return df_zones.collect()[:10:]

# Top 10 ranked zones by passenger volume
def t32_top10_passenger(df_zones):
    # input: output of task 3.2
    df_zones = zones.groupby(["PULocationID_zone"]).agg(pf.sum("total_passengers").alias("passengers")).sort(["passengers"], ascending = False)
    return df_zones.collect()[:10:]

In [0]:
# execute task 3.2
zones = t32_summarise_zones_pairs(graph)

top10_trips     = t32_top10_trips(zones)
top10_profit    = t32_top10_profit(zones)
top10_passenger = t32_top10_passenger(zones)

In [0]:
# use 'display()' or return a pandas DataFrame for 'pretty' output
display(top10_trips)

PULocationID_zone,trips
Upper East Side South,119710
Upper East Side North,101332
Penn Station/Madison Sq West,95537
Midtown Center,91937
Murray Hill,86045
Midtown East,83518
Lincoln Square East,79065
Lenox Hill West,72383
Clinton East,72335
Upper West Side South,72318


In [0]:
# use 'display()' return a pandas DataFrame for 'pretty' output
display(top10_profit)

PULocationID_zone,average
Queensboro Hill,259.1038888888889
Baisley Park,193.0674766995614
Prospect Park,169.1752314814815
Eastchester,107.23302083333334
Saint Michaels Cemetery/Woodside,73.3146103896104
Van Cortlandt Village,67.23423333333334
Crotona Park,62.13000000000002
Riverdale/North Riverdale/Fieldston,62.017999999999994
Flatbush/Ditmas Park,59.37259166666667
Bellerose,48.46958333333333


In [0]:
# use 'display()' or return a pandas DataFrame for 'pretty' output
display(top10_passenger)

PULocationID_zone,passengers
Upper East Side South,169790.0
Upper East Side North,143782.0
Penn Station/Madison Sq West,136813.0
Midtown Center,134603.0
Murray Hill,123489.0
Midtown East,119419.0
Lincoln Square East,114567.0
Clinton East,106834.0
Upper West Side South,105649.0
East Village,105261.0


## Task 4 - Record the pipeline's execution time

Record the execution time of:

1. the whole pipeline
2. the whole pipeline except task 1.2

on the two tables below, for all dataset sizes: `'S'`, `'M'`, `'L'`, `'XL'`, `'XXL'`, and data formats: `parquet` and `delta`.

Analyse the resulting execution times and comment on the effect of dataset size, dataset format and task complexity (with and without task 1.2) on pipeline performance.

In [0]:
# CHANGE the value of the following arguments to record the pipeline execution times for increasing dataset sizes
SIZE = 'XXL'
DATA_FORMAT = 'delta'
WITH_TASK_12 = True

# Load trips dataset
trips = init_trips(SIZE, DATA_FORMAT)


    Trips dataset loaded!
    ---
      Size: XXL
      Format: delta
      Tables loaded: /FileStore/tables/taxi/delta/taxi-XXL-delta/
      Number of trips (dataset rows): 132,396,785
    


In [0]:
# run and record the resulting execution time shown by databricks (on the cell footer)

# IMPORTANT: this function calls all task functions in order of occurrence. For this code to run without errors, you have to load into memory all of the previous task-specific functions, even if you haven't implemented these yet.
pipeline(trips, with_task_12 = WITH_TASK_12)

[[Row(PULocationID_zone='Upper East Side South', trips=119710),
  Row(PULocationID_zone='Upper East Side North', trips=101332),
  Row(PULocationID_zone='Penn Station/Madison Sq West', trips=95537),
  Row(PULocationID_zone='Midtown Center', trips=91937),
  Row(PULocationID_zone='Murray Hill', trips=86045),
  Row(PULocationID_zone='Midtown East', trips=83518),
  Row(PULocationID_zone='Lincoln Square East', trips=79065),
  Row(PULocationID_zone='Lenox Hill West', trips=72383),
  Row(PULocationID_zone='Clinton East', trips=72335),
  Row(PULocationID_zone='Upper West Side South', trips=72318)],
 [Row(PULocationID_zone='Queensboro Hill', average=259.1038888888889),
  Row(PULocationID_zone='Baisley Park', average=193.0674766995614),
  Row(PULocationID_zone='Prospect Park', average=169.1752314814815),
  Row(PULocationID_zone='Eastchester', average=107.23302083333334),
  Row(PULocationID_zone='Saint Michaels Cemetery/Woodside', average=73.3146103896104),
  Row(PULocationID_zone='Van Cortlandt V

_Table 1. Pipeline performance for `parquet` format._

| metric                      | S    | M    | L    | XL   | XXL  |
|-----------------------------|------|------|------|------|------|
| rows (M)                    |  2.898 |  15.57 |  41.95 |  90.44 |  132.4 |
| execution time   (w/o 1.2)  | 3.30 sec | 3.57 sec | 3.72 sec | 4.06 sec | 4.36 sec |
| execution time              | 5.9 sec | 12.73 sec | 33.14 sec | 1.48 minute | 1.77 minute |
| sec / 1M records (w/o 1.2)  | 1.14  | 0.23  | 0.09 | 0.05 | 0.04 |
| sec / 1M records            | 2.04  | 0.82 | 0.79 | 0.98  | 0.80 |

_Table 2. Pipeline performance for `delta` format._

| metric                      | S    | M    | L    | XL   | XXL  |
|-----------------------------|------|------|------|------|------|
| rows (M)                    |  2.898 |  15.57 |  41.95 |  90.44 |  132.4 |
| execution time   (w/o 1.2)  | 3.52 sec | 3.60 sec | 3.72 sec | 3.76 sec | 4.28 sec |
| execution time              | 5.28 sec | 6.61 sec | 8.19 sec | 8.25 sec | 8.99 sec |
| sec / 1M records (w/o 1.2)  | 1.22  | 0.23 | 0.09 | 0.009 | 0.032 |
| sec / 1M records            | 1.83 | 0.43 | 0.20 | 0.091 | 0.07 |


The above piepline was run on the cluster `CSC8641`

Following things can be noted from the above performance table:

1. The execution time for the pipeline (without task 1.2) is much less than that of the execution time of pipeline (with task 1.2)
2. As we can see, as size of the dataset increases the execution time also increases which shows `linearity`.
3. It can be seen from the above table that execution time required for `parquet` format is higher than `delta` file format.
4. The execution time required for `parquet` format with task 1.2 for `XXL data size` is almost `1.77 minutes` whereas for `delta` format it's `below 9 seconds`.
5. It means the `delta` file format is feasible to use for less execution time.